### **DimUser**

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

import os
import sys

project_pth = os.path.join(os.getcwd(),'..','..')
sys.path.append(project_pth)

from utils.transformation import reusable

In [0]:
df = spark.read.format("parquet")\
          .load("abfss://bronze@kobinaazureproject.dfs.core.windows.net/DimUser")

In [0]:
display(df)

#### **AUTOLOADER**

In [0]:
df_user = spark.readStream.format("cloudFiles")\
  .option("cloudFiles.format", "parquet")\
  .option("cloudFiles.schemaLocation", "abfss://silver@kobinaazureproject.dfs.core.windows.net/DimUser/schema")\
  .option("cloudFiles.schemaEvolutionMode", "addNewColumns")\
  .load("abfss://bronze@kobinaazureproject.dfs.core.windows.net/DimUser")


In [0]:
display(df_user, checkpointLocation="abfss://silver@kobinaazureproject.dfs.core.windows.net/DimUser/display_checkpoint")

In [0]:
###df_user_obj = reusable()
### df_user = df_user_obj.dropColumns(df_user,['_rescued_data'])
### display(df_user)

In [0]:
df_user.writeStream.format("delta")\
            .outputMode("append")\
            .option("checkpointLocation","abfss://silver@kobinaazureproject.dfs.core.windows.net/DimUser/checkpoint")\
            .trigger(once=True)\
            .option("path", "abfss://silver@kobinaazureproject.dfs.core.windows.net/DimUser/data")\
            .toTable("spotify_cata.silver.DimUser")           

### **DimArtist**

In [0]:
df_art = (spark.readStream.format("cloudFiles")
  .option("cloudFiles.format", "parquet")
  .option("cloudFiles.schemaLocation", "abfss://silver@kobinaazureproject.dfs.core.windows.net/DimArtist/schema")
  .option("cloudFiles.schemaEvolutionMode", "addNewColumns")
  .load("abfss://bronze@kobinaazureproject.dfs.core.windows.net/DimArtist")
)

In [0]:
display(df_art, checkpointLocation="abfss://silver@kobinaazureproject.dfs.core.windows.net/DimArtist/display_checkpoint")

In [0]:
###df_art_obj = reusable()
### df_art = df_art_obj.dropColumns(df_art,['_rescued_data'])
### display(df_art)

In [0]:
df_art.writeStream.format("delta")\
            .outputMode("append")\
            .option("checkpointLocation","abfss://silver@kobinaazureproject.dfs.core.windows.net/DimArtist/checkpoint")\
            .trigger(once=True)\
            .option("path", "abfss://silver@kobinaazureproject.dfs.core.windows.net/DimArtist/data")\
            .toTable("spotify_cata.silver.DimArtist")      

### **DimTrack**

In [0]:
df_track = spark.readStream.format("cloudFiles")\
  .option("cloudFiles.format", "parquet")\
  .option("cloudFiles.schemaLocation", "abfss://silver@kobinaazureproject.dfs.core.windows.net/DimTrack/schema")\
  .option("cloudFiles.schemaEvolutionMode", "addNewColumns")\
  .load("abfss://bronze@kobinaazureproject.dfs.core.windows.net/DimTrack")


In [0]:
display(df_track, checkpointLocation="abfss://silver@kobinaazureproject.dfs.core.windows.net/DimTrack/display_checkpoint")

In [0]:
##df_track = df_track.withColumn("durationFlag",when(col('duration_sec')<150,"low")\
##                         .when(col('duration_sec')<300,"medium")\
##                         .otherwise("high"))

##df_track = df_track.withColumn("track_name",regexp_replace(col('track_name'),'-',''))
##display(df_track,output_mode='append', checkpointLocation="abfss://silver@kobinaazureproject.dfs.core.windows.net/DimTrack/##display_checkpoint")

In [0]:
df_track.writeStream.format("delta")\
            .outputMode("append")\
            .option("checkpointLocation","abfss://silver@kobinaazureproject.dfs.core.windows.net/DimTrack/checkpoint")\
            .trigger(once=True)\
            .option("path", "abfss://silver@kobinaazureproject.dfs.core.windows.net/DimTrack/data")\
            .toTable("spotify_cata.silver.DimTrack") 

### **DimDate**

In [0]:
df_date = spark.readStream.format("cloudFiles")\
  .option("cloudFiles.format", "parquet")\
  .option("cloudFiles.schemaLocation", "abfss://silver@kobinaazureproject.dfs.core.windows.net/DimDate/schema")\
  .option("cloudFiles.schemaEvolutionMode", "addNewColumns")\
  .load("abfss://bronze@kobinaazureproject.dfs.core.windows.net/DimDate")


In [0]:
df_date.writeStream.format("delta")\
            .outputMode("append")\
            .option("checkpointLocation","abfss://silver@kobinaazureproject.dfs.core.windows.net/DimDate/checkpoint")\
            .trigger(once=True)\
            .option("path", "abfss://silver@kobinaazureproject.dfs.core.windows.net/DimDate/data")\
            .toTable("spotify_cata.silver.DimDate") 

### **FactStream**

In [0]:
df_fact = spark.readStream.format("cloudFiles")\
  .option("cloudFiles.format", "parquet")\
  .option("cloudFiles.schemaLocation", "abfss://silver@kobinaazureproject.dfs.core.windows.net/FactStream/schema")\
  .option("cloudFiles.schemaEvolutionMode", "addNewColumns")\
  .load("abfss://bronze@kobinaazureproject.dfs.core.windows.net/FactStream")

In [0]:
display(df_fact, checkpointLocation="abfss://silver@kobinaazureproject.dfs.core.windows.net/FactStream/display_checkpoint")

In [0]:
df_fact.writeStream.format("delta")\
            .outputMode("append")\
            .option("checkpointLocation","abfss://silver@kobinaazureproject.dfs.core.windows.net/FactStream/checkpoint")\
            .trigger(once=True)\
            .option("path", "abfss://silver@kobinaazureproject.dfs.core.windows.net/FactStream/data")\
            .toTable("spotify_cata.silver.FactStream")